In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/asba-nlim/airy_test_nlim.csv
/kaggle/input/asba-nlim/airy_test_nlim2.csv
/kaggle/input/asba-nlim/rest_test_nlim.csv
/kaggle/input/asba-nlim/car_train_nlim.csv
/kaggle/input/asba-nlim/rest_train_nlim.csv
/kaggle/input/asba-nlim/airy_train_nlim.csv
/kaggle/input/asba-nlim/car_test_nlim.csv
/kaggle/input/corpus-sentence-piece-asba/airy_sp10m.cased.v3.model
/kaggle/input/corpus-sentence-piece-asba/airy_rest_sp10m.cased.v3.model
/kaggle/input/corpus-sentence-piece-asba/car_sp10m.cased.v3.vocab
/kaggle/input/corpus-sentence-piece-asba/car_rest_sp10m.cased.v3.vocab
/kaggle/input/corpus-sentence-piece-asba/car_rest_corpus.txt
/kaggle/input/corpus-sentence-piece-asba/airy_corpus.txt
/kaggle/input/corpus-sentence-piece-asba/airy_car_sp10m.cased.v3.model
/kaggle/input/corpus-sentence-piece-asba/rest_corpus.txt
/kaggle/input/corpus-sentence-piece-asba/all_sp10m.cased.v3.model
/kaggle/input/corpus-sentence-piece-asba/car_rest_sp10m.cased.v3.model
/kaggle/input/corpus-sentence-piece-as

In [2]:
import numpy as np
import os
import pandas as pd

In [3]:
path_to_data='/kaggle/input/corpus-sentence-piece-asba'
# os.listdir(path_to_data)###

#### Making data for language model


We will be needing one sentence per line and blank line means end of document. We will be using entire train and test data to make that file.

We merged train and test reviews and combined them *

In [4]:
print (os.listdir(path_to_data))

['airy_sp10m.cased.v3.model', 'airy_rest_sp10m.cased.v3.model', 'car_sp10m.cased.v3.vocab', 'car_rest_sp10m.cased.v3.vocab', 'car_rest_corpus.txt', 'airy_corpus.txt', 'airy_car_sp10m.cased.v3.model', 'rest_corpus.txt', 'all_sp10m.cased.v3.model', 'car_rest_sp10m.cased.v3.model', 'airy_rest_corpus.txt', 'rest_sp10m.cased.v3.vocab', 'airy_sp10m.cased.v3.vocab', 'all_sp10m.cased.v3.vocab', 'airy_car_sp10m.cased.v3.vocab', 'airy_rest_sp10m.cased.v3.vocab', 'all_corpus.txt', 'rest_sp10m.cased.v3.model', 'car_corpus.txt', 'car_sp10m.cased.v3.model']


In [5]:
##clone xlnet official repository
!git clone https://github.com/arfaghif/malaya

Cloning into 'malaya'...
remote: Enumerating objects: 17730, done.
remote: Counting objects: 100% (17730/17730), done.
remote: Compressing objects: 100% (4511/4511), done.
remote: Total 17730 (delta 12831), reused 17712 (delta 12823), pack-reused 0
Receiving objects: 100% (17730/17730), 122.06 MiB | 27.12 MiB/s, done.
Resolving deltas: 100% (12831/12831), done.


### Using sentence piece model from Google to tokenize text
We will be using  Google's **sentencepiece** model to tokenize the text. 

In [6]:
!pip install sentencepiece

In [7]:
path_to_data

'/kaggle/input/corpus-sentence-piece-asba'

In [8]:
curr_dir=os.getcwd()

In [9]:
curr_dir

'/kaggle/working'

In [10]:
%cd '/kaggle/input/corpus-sentence-piece-asba'


/kaggle/input/corpus-sentence-piece-asba


In [11]:
%cd '/kaggle/working/malaya/pretrained-model/xlnet'

/kaggle/working/malaya/pretrained-model/xlnet


In [12]:
print (os.listdir(path_to_data))

['airy_sp10m.cased.v3.model', 'airy_rest_sp10m.cased.v3.model', 'car_sp10m.cased.v3.vocab', 'car_rest_sp10m.cased.v3.vocab', 'car_rest_corpus.txt', 'airy_corpus.txt', 'airy_car_sp10m.cased.v3.model', 'rest_corpus.txt', 'all_sp10m.cased.v3.model', 'car_rest_sp10m.cased.v3.model', 'airy_rest_corpus.txt', 'rest_sp10m.cased.v3.vocab', 'airy_sp10m.cased.v3.vocab', 'all_sp10m.cased.v3.vocab', 'airy_car_sp10m.cased.v3.vocab', 'airy_rest_sp10m.cased.v3.vocab', 'all_corpus.txt', 'rest_sp10m.cased.v3.model', 'car_corpus.txt', 'car_sp10m.cased.v3.model']


In [13]:
##We will be saving tf records in a folder named experimenttfrecords

In [14]:
# !git pull origin versi3.7

In [15]:
%%writefile prepro_utils.py
# coding=utf-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import unicodedata
import six
from functools import partial


SPIECE_UNDERLINE = '▁'


def printable_text(text):
  """Returns text encoded in a way suitable for print or `tf.logging`."""

  # These functions want `str` for both Python2 and Python3, but in one case
  # it's a Unicode string and in the other it's a byte string.
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text
    elif isinstance(text, unicode):
      return text.encode("utf-8")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


def print_(*args):
  new_args = []
  for arg in args:
    if isinstance(arg, list):
      s = [printable_text(i) for i in arg]
      s = ' '.join(s)
      new_args.append(s)
    else:
      new_args.append(printable_text(arg))
  print(*new_args)


def preprocess_text(inputs, lower=False, remove_space=True, keep_accents=False):
  if remove_space:
    outputs = ' '.join(inputs.strip().split())
  else:
    outputs = inputs
  outputs = outputs.replace("``", '"').replace("''", '"')

  if six.PY2 and isinstance(outputs, str):
    outputs = outputs.decode('utf-8')

  if not keep_accents:
    outputs = unicodedata.normalize('NFKD', outputs)
    outputs = ''.join([c for c in outputs if not unicodedata.combining(c)])
  if lower:
    outputs = outputs.lower()

  return outputs


def encode_pieces(sp_model, text, return_unicode=True, sample=False):
  # return_unicode is used only for py2

  # note(zhiliny): in some systems, sentencepiece only accepts str for py2
  if six.PY2 and isinstance(text, unicode):
    text = text.encode('utf-8')

  if not sample:
    pieces = sp_model.EncodeAsPieces(text)
  else:
    pieces = sp_model.SampleEncodeAsPieces(text, 64, 0.1)
  new_pieces = []
  for piece in pieces:
    if len(piece) > 1 and piece[-1] == ',' and piece[-2].isdigit():
      cur_pieces = sp_model.EncodeAsPieces(
          piece[:-1].replace(SPIECE_UNDERLINE, ''))
      if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
        if len(cur_pieces[0]) == 1:
          cur_pieces = cur_pieces[1:]
        else:
          cur_pieces[0] = cur_pieces[0][1:]
      cur_pieces.append(piece[-1])
      new_pieces.extend(cur_pieces)
    else:
      new_pieces.append(piece)

  # note(zhiliny): convert back to unicode for py2
  if six.PY2 and return_unicode:
    ret_pieces = []
    for piece in new_pieces:
      if isinstance(piece, str):
        piece = piece.decode('utf-8')
      ret_pieces.append(piece)
    new_pieces = ret_pieces

  return new_pieces


def encode_ids(sp_model, text, sample=False):
  pieces = encode_pieces(sp_model, text, return_unicode=False, sample=sample)
  ids = [sp_model.PieceToId(piece) for piece in pieces]
  return ids


if __name__ == '__main__':
  import sentencepiece as spm

  sp = spm.SentencePieceProcessor()
  sp.load('sp10m.uncased.v3.model')

  print_(u'I was born in 2000, and this is falsé.')
  print_(u'ORIGINAL', sp.EncodeAsPieces(u'I was born in 2000, and this is falsé.'))
  print_(u'OURS', encode_pieces(sp, u'I was born in 2000, and this is falsé.'))
  print(encode_ids(sp, u'I was born in 2000, and this is falsé.'))
  print_('')
  prepro_func = partial(preprocess_text, lower=True)
  print_(prepro_func('I was born in 2000, and this is falsé.'))
  print_('ORIGINAL', sp.EncodeAsPieces(prepro_func('I was born in 2000, and this is falsé.')))
  print_('OURS', encode_pieces(sp, prepro_func('I was born in 2000, and this is falsé.')))
  print(encode_ids(sp, prepro_func('I was born in 2000, and this is falsé.')))
  print_('')
  print_('I was born in 2000, and this is falsé.')
  print_('ORIGINAL', sp.EncodeAsPieces('I was born in 2000, and this is falsé.'))
  print_('OURS', encode_pieces(sp, 'I was born in 2000, and this is falsé.'))
  print(encode_ids(sp, 'I was born in 2000, and this is falsé.'))
  print_('')
  print_('I was born in 92000, and this is falsé.')
  print_('ORIGINAL', sp.EncodeAsPieces('I was born in 92000, and this is falsé.'))
  print_('OURS', encode_pieces(sp, 'I was born in 92000, and this is falsé.'))
  print(encode_ids(sp, 'I was born in 92000, and this is falsé.'))

Overwriting prepro_utils.py


In [16]:
!python data_utils.py \
   --use_tpu=False \
   --bsz_per_host=8 \
   --num_core_per_host=1 \
   --uncased=False \
   --seq_len=128 \
   --reuse_len=64 \
   --input_glob="/kaggle/input/corpus-sentence-piece-asba/car_corpus.txt" \
   --save_dir="/kaggle/working/c_experimenttfrecords" \
   --sp_path="/kaggle/input/corpus-sentence-piece-asba/car_sp10m.cased.v3.model" \
   --mask_alpha=6 --mask_beta=1 --num_predict=50 --bi_data=False

2023-01-30 11:30:25.829731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 11:30:25.830742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 11:30:26.008987: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 11:30:26.009910: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 11:30:26.010766: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Now, it's time to train the model.

In [17]:
!wget https://f000.backblazeb2.com/file/malaya-model/bert-bahasa/xlnet-base-29-03-2020.tar.gz

--2023-01-30 11:30:28--  https://f000.backblazeb2.com/file/malaya-model/bert-bahasa/xlnet-base-29-03-2020.tar.gz
Resolving f000.backblazeb2.com (f000.backblazeb2.com)... 104.153.233.177
Connecting to f000.backblazeb2.com (f000.backblazeb2.com)|104.153.233.177|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 1275743608 (1.2G) [application/octet-stream]
Saving to: ‘xlnet-base-29-03-2020.tar.gz’

xlnet-base-29-03-20 100%[===================>]   1.19G  42.4MB/s    in 33s     

2023-01-30 11:31:02 (36.9 MB/s) - ‘xlnet-base-29-03-2020.tar.gz’ saved [1275743608/1275743608]



In [18]:
!tar -xvf xlnet-base-29-03-2020.tar.gz

xlnet-base-29-03-2020/
xlnet-base-29-03-2020/sp10m.cased.v9.vocab
xlnet-base-29-03-2020/sp10m.cased.v9.model
xlnet-base-29-03-2020/config.json
xlnet-base-29-03-2020/model.ckpt-300000.data-00000-of-00001
xlnet-base-29-03-2020/model.ckpt-300000.index
xlnet-base-29-03-2020/model.ckpt-300000.meta


###  We will be training our language model from checkpoint. Checkpoint is the model downloaded above.

We will save the newly trained model inside the directoy experimenttfrecords created above. If we omit init_checkpoint argument we will get freshly trained language model.

In [19]:
!python train_gpu.py \
   --corpus_info_path="/kaggle/working/c_experimenttfrecords/corpus_info.json" \
   --model_dir="/kaggle/working/c_experimenttfrecords/model" \
   --record_info_dir="/kaggle/working/c_experimenttfrecords/tfrecords" \
   --train_batch_size=8 \
   --seq_len=128 \
   --reuse_len=64 \
   --mem_len=384 \
   --perm_size=64 \
   --untie_r=True \
   --mask_alpha=6 \
   --mask_beta=1 \
   --num_predict=50 \
   --train_steps=40 \
   --iterations=10 \
   --uncased=False \
   --bi_data=False \
   --save_steps=2 \
   --d_head=64 \
   --d_inner=3072 \
   --d_model=768 \
   --ff_activation=gelu \
   --n_head=12 \
   --n_layer=12 \
   --n_token=32000 \
   --init_checkpoint='xlnet-base-29-03-2020/model.ckpt-300000'

2023-01-30 11:31:21.978347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 11:31:21.979306: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 11:31:21.988968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 11:31:21.989861: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 11:31:21.990708: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [20]:
%cd '/kaggle/working'

/kaggle/working


In [21]:
!mkdir '/kaggle/working/c_experimenttfrecords/model/c_pytorch_model'

#### Now our retrained language model is ready. Cheers it's been achieved after lot of effforts. Now we will use pytorch transformers(God keep them happy) transfomer-cli  to change this Tensorflow model to pytorch model

In [22]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 125165, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 125165 (delta 47), reused 59 (delta 40), pack-reused 125088
Receiving objects: 100% (125165/125165), 119.73 MiB | 26.49 MiB/s, done.
Resolving deltas: 100% (93942/93942), done.


Converting tensorflow model into pytorch model

In [23]:
%cd 'transformers'

/kaggle/working/transformers


In [24]:
# !export TRANSFO_XL_CHECKPOINT_PATH='/content/drive/My Drive/experimenttfrecords/model/model.ckpt'
# !export TRANSFO_XL_CONFIG_PATH='/content/drive/My Drive/experimenttfrecords/model/config.json'
# !export PYTORCH_MODELPATH='/content/drive/My Drive/experimenttfrecords/pytorch_model'

In [25]:
!pip install transformers

In [26]:
!transformers-cli convert \
  --model_type xlnet  \
  --tf_checkpoint="/kaggle/working/c_experimenttfrecords/model/model.ckpt" \
  --config="/kaggle/working/c_experimenttfrecords/model/config.json" \
  --pytorch_dump_output="/kaggle/working/c_experimenttfrecords/model/c_pytorch_model"

Loading TF weight beta1_power with shape []
Loading TF weight beta2_power with shape []
Loading TF weight global_step with shape []
Loading TF weight model/lm_loss/bias with shape [32000]
Loading TF weight model/lm_loss/bias/Adam with shape [32000]
Loading TF weight model/lm_loss/bias/Adam_1 with shape [32000]
Loading TF weight model/transformer/layer_0/ff/layer_1/bias with shape [3072]
Loading TF weight model/transformer/layer_0/ff/layer_1/bias/Adam with shape [3072]
Loading TF weight model/transformer/layer_0/ff/layer_1/bias/Adam_1 with shape [3072]
Loading TF weight model/transformer/layer_0/ff/layer_1/kernel with shape [768, 3072]
Loading TF weight model/transformer/layer_0/ff/layer_1/kernel/Adam with shape [768, 3072]
Loading TF weight model/transformer/layer_0/ff/layer_1/kernel/Adam_1 with shape [768, 3072]
Loading TF weight model/transformer/layer_0/ff/layer_2/bias with shape [768]
Loading TF weight model/transformer/layer_0/ff/layer_2/bias/Adam with shape [768]
Loading TF weigh

In [27]:
!rm -r "/kaggle/working/transformers"
!rm -r "/kaggle/working/malaya"

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


#### DONE THANKYOU
You now have xlnet language model finetuned on your dataset.